In [6]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import random

# Load dataset
file_path = "/content/HeartDiseaseTrain-Test.csv"  # Corrected file path
df = pd.read_csv(file_path)

# Display first few rows
print(df.head())

# Check data types and missing values
print(df.info())
print(df.isnull().sum())

# Drop non-numeric columns
non_numeric_cols = df.select_dtypes(include=['object']).columns
df = df.drop(columns=non_numeric_cols)  # Remove text-based columns

# Handle missing values (only for numeric columns)
df.fillna(df.mean(numeric_only=True), inplace=True)

# Select 'age' and 'cholestoral' as features, 'target' as output (converted to binary)
X1 = df['age'].values
X2 = df['cholestoral'].values  # Fixed column name
y = (df['target'] > 0).astype(int).values  # Convert 'target' > 0 to 1 (heart disease)

# Normalize features using Min-Max Scaling
X1_min, X1_max = min(X1), max(X1)
X2_min, X2_max = min(X2), max(X2)

X1 = (X1 - X1_min) / (X1_max - X1_min)
X2 = (X2 - X2_min) / (X2_max - X2_min)

print(f"Processed Data:\nX1 (Age): {X1[:5]}\nX2 (Cholesterol): {X2[:5]}\ny (Target): {y[:5]}")


   age     sex chest_pain_type  resting_blood_pressure  cholestoral  \
0   52    Male  Typical angina                     125          212   
1   53    Male  Typical angina                     140          203   
2   70    Male  Typical angina                     145          174   
3   61    Male  Typical angina                     148          203   
4   62  Female  Typical angina                     138          294   

      fasting_blood_sugar               rest_ecg  Max_heart_rate  \
0    Lower than 120 mg/ml  ST-T wave abnormality             168   
1  Greater than 120 mg/ml                 Normal             155   
2    Lower than 120 mg/ml  ST-T wave abnormality             125   
3    Lower than 120 mg/ml  ST-T wave abnormality             161   
4  Greater than 120 mg/ml  ST-T wave abnormality             106   

  exercise_induced_angina  oldpeak        slope vessels_colored_by_flourosopy  \
0                      No      1.0  Downsloping                           Two   
1 

In [7]:
# Manually split into 80% training and 20% testing
split_ratio = 0.8
split_index = int(len(X1) * split_ratio)

X1_train, X1_test = X1[:split_index], X1[split_index:]
X2_train, X2_test = X2[:split_index], X2[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [8]:
# Sigmoid Activation Function
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

# Derivative of Sigmoid
def sigmoid_derivative(x):
    return x * (1 - x)

# ReLU Activation Function
def relu(x):
    return max(0, x)

# Derivative of ReLU
def relu_derivative(x):
    return 1 if x > 0 else 0


In [9]:
# Random Weight Initialization
random.seed(42)
w1 = random.uniform(-1, 1)  # Weight for X1 to Hidden Neuron 1
w2 = random.uniform(-1, 1)  # Weight for X2 to Hidden Neuron 1
w3 = random.uniform(-1, 1)  # Weight for X1 to Hidden Neuron 2
w4 = random.uniform(-1, 1)  # Weight for X2 to Hidden Neuron 2

b1 = random.uniform(-1, 1)  # Bias for Hidden Neuron 1
b2 = random.uniform(-1, 1)  # Bias for Hidden Neuron 2

w5 = random.uniform(-1, 1)  # Weight from Hidden Neuron 1 to Output
w6 = random.uniform(-1, 1)  # Weight from Hidden Neuron 2 to Output

b_out = random.uniform(-1, 1)  # Bias for Output Neuron


In [10]:
def forward_propagation(x1, x2):
    # Hidden Layer
    z1 = w1 * x1 + w2 * x2 + b1
    a1 = relu(z1)  # Activation for Neuron 1

    z2 = w3 * x1 + w4 * x2 + b2
    a2 = relu(z2)  # Activation for Neuron 2

    # Output Layer
    z_out = w5 * a1 + w6 * a2 + b_out
    a_out = sigmoid(z_out)  # Output Activation (Sigmoid)

    return a1, a2, a_out


In [11]:
def backpropagation(x1, x2, y_true, a1, a2, a_out, learning_rate=0.01):
    global w1, w2, w3, w4, w5, w6, b1, b2, b_out

    # Compute Output Error
    error = a_out - y_true

    # Compute Gradients for Output Layer
    dz_out = error * sigmoid_derivative(a_out)
    dw5 = dz_out * a1
    dw6 = dz_out * a2
    db_out = dz_out

    # Compute Gradients for Hidden Layer
    dz1 = dz_out * w5 * relu_derivative(a1)
    dz2 = dz_out * w6 * relu_derivative(a2)

    dw1 = dz1 * x1
    dw2 = dz1 * x2
    db1 = dz1

    dw3 = dz2 * x1
    dw4 = dz2 * x2
    db2 = dz2

    # Update Weights and Biases
    w1 -= learning_rate * dw1
    w2 -= learning_rate * dw2
    w3 -= learning_rate * dw3
    w4 -= learning_rate * dw4
    w5 -= learning_rate * dw5
    w6 -= learning_rate * dw6

    b1 -= learning_rate * db1
    b2 -= learning_rate * db2
    b_out -= learning_rate * db_out


In [14]:
def predict(age, chol):
    # Normalize Input
    age = (age - X1_min) / (X1_max - X1_min)
    chol = (chol - X2_min) / (X2_max - X2_min)

    _, _, a_out = forward_propagation(age, chol)
    return 1 if a_out > 0.5 else 0

# Example Prediction
new_age = 55
new_chol = 240
predicted_label = predict(new_age, new_chol)
print(f"Predicted Heart Disease: {predicted_label}")


Predicted Heart Disease: 0


In [12]:
def train(epochs=1000, learning_rate=0.01):
    for epoch in range(epochs):
        total_loss = 0
        for i in range(len(X1_train)):
            a1, a2, a_out = forward_propagation(X1_train[i], X2_train[i])
            backpropagation(X1_train[i], X2_train[i], y_train[i], a1, a2, a_out, learning_rate)
            total_loss += (a_out - y_train[i])**2  # MSE Loss

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {total_loss / len(X1)}")

train(epochs=1000, learning_rate=0.01)


Epoch 0, Loss: 0.20132588920941757
Epoch 100, Loss: 0.18279252434140217
Epoch 200, Loss: 0.18251581577390558
Epoch 300, Loss: 0.1824918079801892
Epoch 400, Loss: 0.18250125948831766
Epoch 500, Loss: 0.18249034044644705
Epoch 600, Loss: 0.18247891479845377
Epoch 700, Loss: 0.18247132419798454
Epoch 800, Loss: 0.18246584573164418
Epoch 900, Loss: 0.18246240247467774


In [15]:
# Evaluate on test set
y_pred_test = [predict(X1_test[i] * (X1_max - X1_min) + X1_min,
                       X2_test[i] * (X2_max - X2_min) + X2_min) for i in range(len(X1_test))]

# Calculate Accuracy
accuracy = sum(y_pred_test[i] == y_test[i] for i in range(len(y_test))) / len(y_test)
print(f"Accuracy on Test Data: {accuracy:.4f}")


Accuracy on Test Data: 0.5902
